In [8]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [1]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [2]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [60]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [6]:
contr = [1]*10 + [0]*24
test = [1]*4 +[0]*12

In [9]:
print "95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(contr, test)

95% confidence interval for a difference between proportions: [-0.217558, 0.305793]


In [17]:
print "p-value: %f" % round(proportions_diff_z_test(proportions_diff_z_stat_ind(contr, test), 'greater'),4)

p-value: 0.372900


Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

логистическая регрессия по признакам X_1,X_2,X_3X

логистическая регрессия по признакам X_4,X_5,X_6

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5\times10^{-8}5.5×10 
−8
 , нужно ввести 8).

In [18]:
data = pd.read_csv("banknotes.txt",sep='\t')

In [54]:
from sklearn.cross_validation import train_test_split 
from sklearn.linear_model import LogisticRegression

In [49]:
x_train , x_test, y_train, y_test = train_test_split(data.drop("real",axis=1),data.real,test_size = 0.25,random_state =1)

In [38]:
x_train.iloc[:,0:3].head()

,X1,X2,X3
98,215.1,130.0,129.8
123,215.1,130.7,130.4
119,214.4,130.2,129.9
53,215.4,130.2,130.2
33,215.6,130.4,130.1


In [55]:
lg_123 = LogisticRegression().fit(x_train.iloc[:,0:3],y_train)
lg_456 = LogisticRegression().fit(x_train.iloc[:,3:6],y_train)

In [56]:
pred_123 = lg_123.predict(x_test.iloc[:,0:3])
pred_456 = lg_456.predict(x_test.iloc[:,3:6])

In [58]:
false_123 = [1 if i != j else 0 for i,j in zip(pred_123,y_test)] 
false_456 = [1 if i != j else 0 for i,j in zip(pred_456,y_test)] 

In [65]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [62]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [63]:
print "p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(false_123, false_456))

p-value: 0.003297


In [68]:
print "95%% confidence interval for a difference between proportions: [%f, %f]" \
      % proportions_diff_confint_rel(false_123, false_456)

95% confidence interval for a difference between proportions: [0.059945, 0.300055]


In [69]:
round(0.059945,4)

0.0599

Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [72]:
z = (541.4 - 525)/(100./np.sqrt(100))

In [73]:
z

1.6399999999999977

In [74]:
p = 1 - scipy.stats.norm.cdf(z)

In [76]:
round(p,4)

0.0505

Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [77]:
z = (541.5 - 525)/(100./np.sqrt(100))
p = 1 - scipy.stats.norm.cdf(z)
round(p,4)

0.0495